In [1]:
"""import os
import pickle
import sys

import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import tabulate
from autorank import autorank, create_report, plot_stats
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator"""
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymer4.models import Lmer
from scipy import stats
from scipy.stats import rankdata
from dataset_management import get_dataset
from models import model

pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

In [9]:
algorithm="algorithm";benchmark="bench_prior";seed="seed";time="used_fidelity";loss="norm_regret"

def group(row):
    return "1-5" if row[time] <= 5 else "6-10" if row[time] <= 10 else "11-15" if row[time] <= 15 else "16-20" if row[time] <= 20 else "21-25"


dataset=get_dataset(dataset_name="fig5_24_at25_bad_intsteps")
dataset=dataset.loc[(dataset["prior"]=="bad")&(dataset[time]>5)]
dataset[f"{time}_group"]=dataset.apply(group,axis=1)
mod=model(formula=f"{loss}~{algorithm}:{time}_group+(1|{benchmark})",data=dataset,factor_list=[f"{time}_group",algorithm])
print(mod.summary())
mod2=model(formula=f"{loss}~{algorithm}+{time}_group+(1|{benchmark})",data=dataset,factor_list=[f"{time}_group",algorithm])
print(mod2.summary())
mod3=model(formula=f"{loss}~{algorithm}:{time}_group+{time}_group+(1|{benchmark})",data=dataset,factor_list=[f"{time}_group",algorithm])
print(mod3.summary())

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: norm_regret~algorithm:used_fidelity_group+(1|bench_prior)

Family: gaussian	 Inference: parametric

Number of observations: 34200	 Groups: {'bench_prior': 12.0}

Log-likelihood: -12675.308 	 AIC: 25378.616

Random effects:

                    Name    Var    Std
bench_prior  (Intercept)  0.333  0.577
Residual                  0.122  0.350

No random effect correlations specified

Fixed effects:

                                      Estimate  2.5_ci  97.5_ci     SE        DF  T-stat  P-val  Sig
(Intercept)                              0.292  -0.035    0.619  0.167     12.04   1.750  0.105     
algorithmPB:used_fidelity_group6-10      0.172   0.154    0.191  0.010  34188.00  17.990  0.000  ***
algorithmRS:used_fidelity_group6-10      0.227   0.208    0.246  0.010  34188.00  23.698  0.000  ***
algorithmHB:used_fidelity_group6-10      0.132   0.114    0.151  0.010  34188.00  13.802  0.000  ***
algorithmPB:used_fidelity_gro

In [10]:
post_hocs=mod.post_hoc(marginal_vars=algorithm,grouping_vars=f"{time}_group")
print(post_hocs)
print(mod2.post_hoc(marginal_vars=algorithm,grouping_vars=f"{time}_group"))
print(mod3.post_hoc(marginal_vars=algorithm,grouping_vars=f"{time}_group"))

P-values adjusted by tukey method for family of 3 estimates
(   algorithm used_fidelity_group  Estimate  2.5_ci  97.5_ci     SE      DF
1         PB                6-10     0.464   0.101    0.828  0.167  12.031
2         RS                6-10     0.519   0.156    0.882  0.167  12.031
3         HB                6-10     0.424   0.061    0.787  0.167  12.031
4         PB               11-15     0.342  -0.021    0.705  0.167  12.031
5         RS               11-15     0.413   0.050    0.777  0.167  12.031
6         HB               11-15     0.359  -0.004    0.723  0.167  12.031
7         PB               16-20     0.288  -0.075    0.652  0.167  12.031
8         RS               16-20     0.367   0.003    0.730  0.167  12.031
9         HB               16-20     0.307  -0.056    0.670  0.167  12.031
10        PB               21-25     0.255  -0.108    0.618  0.167  12.040
11        RS               21-25     0.345  -0.018    0.708  0.167  12.040
12        HB               21-25     0.